Keynote of this lab:
<ol>
    <li>Introduce Langchain</li>
    <li>Introduce chain</li>
    <li>Let them remember what we said!!</li>
</ol>

used packages

In [ ]:
#connect llm
import ollama
#from ollama import chat
#from ollama import ChatResponse
#from ollama import Client

#langchain
import langchain
import langgraph
#import OllamaLLM

#MongoDB
import pymongo as pymg
import pprint as pt
import pandas as pd
from datetime import datetime

use LLM to search website for you

In [ ]:
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_ollama.llms import OllamaLLM as ollm
from googlesearch import search as gs

template: str = """
Question: {question}
Based on the Question, list some keywords for the search engine.
Answer: only Keywords.
"""

prompt = cpt.from_template(template)

llm = "llama3.2:latest"

model = ollm(model = llm, base_url = "http://modelserver:11434")

chain = prompt | model

response = chain.invoke({"question": "怎麼能抽淘博陀螺"})

result = gs(response, num_results = 10, unique = True)

for item in result:
    print(item)

simplest way to let LLM remember your conversation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_ollama.llms import OllamaLLM as ollm
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

msg: list = []

template: str = """
Question: {question}
Answer: Written in Chinese.
"""

prompt = cpt.from_template(template)

llm = "qwen2.5:7b"

model = ollm(model = llm, base_url = "http://modelserver:11434")

chain = prompt | model

def communication(question: dict) -> None:
    msg.append(HumanMessage(content = question["question"]))
    response = chain.invoke(msg)
    msg.append(AIMessage(content = response))
    print(response)

communication({"question": "Who is BLG Bin?"})
communication({"question": "How the sun rist affect BLG Bin?"})
communication({"question": "What I have asked"})

Save Conversion to Database (simplest)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_ollama.llms import OllamaLLM as ollm
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import pymongo as pymg 

def chain_log():
    message = ""
    chat_log = collection.find()
    chat_log = list(chat_log)
    if len(chat_log) != 0:
        for chat in chat_log:
            message += f"Human:{chat["user"]}\nAI: {chat["ai"]}\n"
    else:
        message += "AI: This is a new conersion"
    return message        

def update_chain(last) -> None:
    post = db.conversation
    upload = post.insert_one(last)
    upload.inserted_id

def communication(question) -> None:
    response = rag_chain.invoke(question)
    last_conversation = {
        "user": question,
        "ai": response
    }
    update_chain(last_conversation)
    print(response)

log = pymg.MongoClient("mongodb://mongos", 27017)
db = log["llm"]
db.drop_collection("conversation")
collection = db["conversation"]

template: str = """
You are Benson, a Data Science Teacher. 
Here is the conversion {history}
communicate with human in a normal tone.
Question: {question}
"""

prompt = cpt.from_template(template)

llm = "qwen2.5:7b"

model = ollm(model = llm, base_url = "http://modelserver:11434")

rag_chain = (
    {
        "history": lambda x: chain_log(),
        "question": lambda x: RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

question = [
    "Hello, I am Benson, nice to meet you.",
    "I would like to know how the differences between K-means and K-nearest neighbours algorithms are.",
    "Summarise my asked question"
]

for q in question:
    print(f"Question: {q}")
    communication(q)
    print("=" * 87)

Embbedding
<br>
source: <a href="https://zh.moegirl.org.cn/zh-hk/%E5%B4%A9%E5%9D%8F%EF%BC%9A%E6%98%9F%E7%A9%B9%E9%93%81%E9%81%93">game setting</a>

In [ ]:
#path checking
import os

#document preprocessing
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
#from langchain.vectorstores import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter



documents = []

for file in os.listdir("/Code/TrainingDataSet"):
    doc_path = "/Code/TrainingDataSet/" + file
    loader = Docx2txtLoader(doc_path)
    documents.extend(loader.load())


text_splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 2000)
doc_splits = text_splitter.split_documents(documments)



template: str = """
假設你是崩鐵：星穹鐵道裡的一位虛構史學家，你會與一位界外之民進行溝通。
請根據{context}中的資訊加以整理，再用不同的角度及有趣口風回答{question}。
你的回答必須是獨白或者文章的方式呈現，不能使用點列方式。
"""

prompt = cpt.from_template(template)

llm = "qwen2.5:7b"

model = ollm(model = llm, base_url = "http://modelserver:11434", temperature = 0.1)

emded_target = OllamaEmbeddings(
    model = "mxbai-embed-large:latest",
    base_url = "http://modelserver:11434"
)

vector_store = InMemoryVectorStore.from_documents(documents = doc_splits, embedding = emded_target, collection_name = "HSR")

retriever = vector_store.as_retriever()

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

question_bank = [
    "崩鐵啟動！",
    "下個版本有什麼能新角色"
]

for i in question_bank:
    print(f"Question: {i}")
    response = rag_chain.invoke(i)
    print(f"Question: {response}")
    print("=" * 87)